In [21]:
import sys
sys.path.append("./AIMA")
from search import *
from agents import*

In [22]:
class Tpiece(Thing): #Create a movable piece class
    pass

class Mobs(Thing):
    pass

class Oobs(Thing):
    pass

class Board(Environment): #Create an environment like last assignment
    pass

In [23]:
class PSA(SimpleProblemSolvingAgentProgram):
    def __call__(self, board): #AIMA overrided function because it had some things wrong
        self.percept = board.things #percept will just be the list of objects in the board environment
        self.state = self.update_state(self.percept)
        if not self.seq:
            goal = self.formulate_goal(self.state)
            problem = self.formulate_problem(self.state, goal)
            self.seq.append(self.search(problem))
            #This was originally self.seq = blabla in AIMA which obviously gives an error when trying to pop it later wtf AIMA
            if not self.seq:
                return None
        return self.seq.pop(0) #self.seq goes back empty for working the next goal
    
    def update_state(self, percept): #from board.things it builds a tuple of tuples to represent the initial state of the current step
        #Format: (('Identification char', (location)), next object of the percept in the same format)
        #Example: ( ('A', (1, 1)), ('T', (5, 2)) ) <- means a state with agent in 1, 1 and movable piece in 5, 2
        #Output needs to be tuple because the AIMA graph search function needs hashable data structures to add to the explored set
        
        state = {'M': [], 'O': []} #Easier to build the state as a dict
        for p in percept:
            if isinstance(p, Agent):
                state['A'] = p.location
            elif isinstance(p, Tpiece):
                state['T'] = p.location
            elif isinstance(p, Mobs):
                state['M'].append(p.location)
            elif isinstance(p, Oobs):
                state['O'].append(p.location)
            
        temp = {}
        temp['A'] = state['A']
        temp['T'] = state['T']
        temp['M'] = tuple(state['M'])
        temp['O'] = tuple(state['O'])
        
        state = []
        for k in temp.items():
            state.append(k)
                
        state = tuple(state) #convert the state to tuple
        return state

    def formulate_goal(self, state): #returns the desired state of the current step
        goal = [(('A', (4,5)), ('T', (5,5))), (('A', (5,4)), ('T', (5,5)))]
        #Ww gonna test doing everything in one step first
        return goal

    def formulate_problem(self, state, goal): #create a problem object with the initial state and goal state for this step
        problem = PushnPull(state, goal)
        return problem
    
    def search(self, problem): #Uses some search algorithm included in AIMA to solve the problem
        return graph_search(problem, [])
        

In [27]:
class PushnPull(Problem):
    def actions(self, state):
        """Return the actions that can be executed in the given
        state. The result would typically be a list, but if there are
        many actions, consider yielding them one at a time in an
        iterator, rather than building them all at once."""

        possible_actions = []
        state = dict(state)
        agent = state['A']
        treasure = state['T']
        movable = []
        obstacles = []
        movable_objects = []

        if 'M' in state:
            movable = state['M']
        if 'O' in state:
            obstacles =  list(state['O'])

        movable_objects.append(treasure)
        for m in movable:
            movable_objects.append(m)

        # Booleans for movable objects
        movable_up = checkObject(agent, movable_objects, 'up')
        movable_down = checkObject(agent, movable_objects, 'down')
        movable_left = checkObject(agent, movable_objects, 'left')
        movable_right= checkObject(agent, movable_objects, 'right')

        # Booleans for non movable objects
        obstacle_up = checkObject(agent, obstacles, 'up')
        obstacle_down = checkObject(agent, obstacles, 'down')
        obstacle_left = checkObject(agent, obstacles, 'left')
        obstacle_right = checkObject(agent, obstacles, 'right')

        # Booleans for empty spaces for push/pull
        can_push_up = checkPushPull(agent, movable_objects, obstacles, 'up', 'push')
        can_push_down = checkPushPull(agent, movable_objects, obstacles, 'down', 'push')
        can_push_left = checkPushPull(agent, movable_objects, obstacles, 'left', 'push')
        can_push_right = checkPushPull(agent, movable_objects, obstacles, 'right', 'push')

        can_pull_up = checkPushPull(agent, movable_objects, obstacles, 'up', 'pull')
        can_pull_down = checkPushPull(agent, movable_objects, obstacles, 'down', 'pull')
        can_pull_left = checkPushPull(agent, movable_objects, obstacles, 'left', 'pull')
        can_pull_right = checkPushPull(agent, movable_objects, obstacles, 'right', 'pull')

        # Check Agent's MOVES
        if agent[0] > 1 and not movable_up and not obstacle_up:
            possible_actions.append('moveUp')

        if agent[0] < 5 and not movable_down and not obstacle_down:
            possible_actions.append('moveDown')

        if agent[1] > 1 and not movable_left and not obstacle_left:
            possible_actions.append('moveLeft')

        if agent[1] < 5 and not movable_right and not obstacle_right:
            possible_actions.append('moveRight')

        # Check Agent's PUSHES
        if agent[0] > 2 and movable_up and can_push_up:
            possible_actions.append('pushUp')

        if agent[0] < 4 and movable_down and can_push_down:
            possible_actions.append('pushDown')

        if agent[1] > 2 and movable_left and can_push_left:
            possible_actions.append('pushLeft')

        if agent[1] < 4 and movable_right and can_push_right:
            possible_actions.append('pushRight')

        # Check Agent's PULLS
        if agent[0] > 1 and movable_down and can_pull_up:
            possible_actions.append('pullUp')

        if agent[0] < 5 and movable_up and can_pull_down:
            possible_actions.append('pullDown')

        if agent[1] > 1 and movable_right and can_pull_left:
            possible_actions.append('pullLeft')

        if agent[1] < 5 and movable_left and can_pull_right:
            possible_actions.append('pullRight')

        return possible_actions
    
    def result(self, state, action):
        """Return the state that results from executing the given
        action in the given state. The action must be one of
        self.actions(state)."""
        temp = dict(state)
        #converting the state temporarily to a dictionary makes it easier to manip the data, example: {'A': (1,1), 'T': (5, 5)}
        
        if action == "pushRight":
            out = []
            for i in temp['M']:
                if temp['A'][0] == i[0] and temp['A'][1] + 1 == i[1]:
                    out.append((i[0], i[1] + 1))
                else:
                    out.append((i[0], i[1]))
            temp['M'] = out
            if temp['A'][0] == temp['T'][0] and temp['A'][1] + 1 == temp['T'][1]:
                temp['T'] = (temp['T'][0], temp['T'][1] + 1)
            temp['A'] = (temp['A'][0], temp['A'][1] + 1) 
            
        elif action == "pushLeft":
            out = []
            for i in temp['M']:
                if temp['A'][0] == i[0] and temp['A'][1] - 1 == i[1]:
                    out.append((i[0], i[1] - 1))
                else:
                    out.append((i[0], i[1]))
            temp['M'] = out
            if temp['A'][0] == temp['T'][0] and temp['A'][1] - 1 == temp['T'][1]:
                temp['T'] = (temp['T'][0], temp['T'][1] - 1)
            temp['A'] = (temp['A'][0], temp['A'][1] - 1)

        elif action == "pushUp":
            out = []
            for i in temp['M']:
                if temp['A'][0] - 1 == i[0] and temp['A'][1] == i[1]:
                    out.append((i[0] - 1, i[1]))
                else:
                    out.append((i[0], i[1]))
            temp['M'] = out
            if temp['A'][0] - 1 == temp['T'][0] and temp['A'][1] == temp['T'][1]:
                temp['T'] = (temp['T'][0] - 1, temp['T'][1])
            temp['A'] = (temp['A'][0] - 1, temp['A'][1])

        elif action == "pushDown":
            out = []
            for i in temp['M']:
                if temp['A'][0] + 1 == i[0] and temp['A'][1] == i[1]:
                    out.append((i[0] + 1, i[1]))
                else:
                    out.append((i[0], i[1]))
            temp['M'] = out
            if temp['A'][0] + 1 == temp['T'][0] and temp['A'][1] == temp['T'][1]:
                temp['T'] = (temp['T'][0] + 1, temp['T'][1])
            temp['A'] = (temp['A'][0] + 1, temp['A'][1])
            
        elif action == "pullRight":
            out = []
            for i in temp['M']:
                if temp['A'][0] == i[0] and temp['A'][1] - 1 == i[1]:
                    out.append((i[0], i[1] + 1))
                else:
                    out.append((i[0], i[1]))
            temp['M'] = out
            if temp['A'][0] == temp['T'][0] and temp['A'][1] - 1 == temp['T'][1]:
                temp['T'] = (temp['T'][0], temp['T'][1] + 1)
            temp['A'] = (temp['A'][0], temp['A'][1] + 1)
            
        elif action == "pullLeft":
            out = []
            for i in temp['M']:
                if temp['A'][0] == i[0] and temp['A'][1] + 1 == i[1]:
                    out.append((i[0], i[1] - 1))
                else:
                    out.append((i[0], i[1]))
            temp['M'] = out
            if temp['A'][0] == temp['T'][0] and temp['A'][1] + 1 == temp['T'][1]:
                temp['T'] = (temp['T'][0], temp['T'][1] - 1)
            temp['A'] = (temp['A'][0], temp['A'][1] - 1)
        
        elif action == "pullUp":
            out = []
            for i in temp['M']:
                if temp['A'][0] + 1 == i[0] and temp['A'][1] == i[1]:
                    out.append((i[0] - 1, i[1]))
                else:
                    out.append((i[0], i[1]))
            temp['M'] = out
            if temp['A'][0] + 1 == temp['T'][0] and temp['A'][1] == temp['T'][1]:
                temp['T'] = (temp['T'][0] - 1, temp['T'][1])
            temp['A'] = (temp['A'][0] - 1, temp['A'][1])
            
        elif action == "pullDown":
            out = []
            for i in temp['M']:
                if temp['A'][0] - 1 == i[0] and temp['A'][1] == i[1]:
                    out.append((i[0] + 1, i[1]))
                else:
                    out.append((i[0], i[1]))
            temp['M'] = out
            if temp['A'][0] - 1 == temp['T'][0] and temp['A'][1] == temp['T'][1]:
                temp['T'] = (temp['T'][0] + 1, temp['T'][1])
            temp['A'] = (temp['A'][0] + 1, temp['A'][1])
            
        else:
            if action == 'moveRight':
                temp['A'] = (temp['A'][0], temp['A'][1] + 1)
            elif action == 'moveLeft':
                temp['A'] = (temp['A'][0], temp['A'][1] - 1)
            elif action == 'moveUp':
                temp['A'] = (temp['A'][0] - 1, temp['A'][1])
            elif action == 'moveDown':
                temp['A'] = (temp['A'][0] + 1, temp['A'][1])
            #The new state has the agent move accordingly
        
        #This next block simply builds a proper tuple formatted state from the edited dictionary
        out = {}
        out['A'] = temp['A']
        out['T'] = temp['T']
        out['M'] = tuple(temp['M'])
        out['O'] = temp['O']
        
        result = []
        for k in out.items():
            result.append(k)
                
        result = tuple(result)
        return result
    
    
    def goal_test(self, state):
        state = dict(state)
        agent = ('A', state['A'])
        piece = ('T', state['T'])

        for g in self.goal:
            g_agent = g[0]
            g_piece = g[1]
            if g_agent == agent and g_piece == piece:
                return True
        return False

In [25]:
def graph_search(problem, frontier): #This is just a default AIMA search algorithm which is ran from an agent's search method
    """Search through the successors of a problem to find a goal.
    The argument frontier should be an empty queue.
    If two paths reach a state, only use the first one. [Figure 3.7]"""
    frontier.append(Node(problem.initial))
    explored = set()
    while frontier:
        node = frontier.pop()
        if problem.goal_test(node.state):
            return node
        explored.add(node.state)
        frontier.extend(child for child in node.expand(problem)
                        if child.state not in explored and
                        child not in frontier)
    return None

def checkPushPull(agent, movable_objects, obstacles, direction, movement):
    location = list(agent)

    if direction == 'up':
        if movement == 'push':
            location[0] -= 2
        elif movement == 'pull':
            location[0] -= 1
    elif direction == 'down':
        if movement == 'push':
            location[0] += 2
        elif movement == 'pull':
            location[0] += 1
    elif direction == 'left':
        if movement == 'push':
            location[1] -= 2
        elif movement == 'pull':
            location[1] -= 1
    else:
        if movement == 'push':
            location[1] += 2
        elif movement == 'pull':
            location[1] += 1

    if location[0] > 5 or location[0] < 1 or location[1] > 5 or location[1] < 1:
        return False

    for ob in obstacles:
        if tuple(location) == ob:
            return False

    for mo in movable_objects:
        if tuple(location) == mo:
            return False
    return True

def checkObject(agent, objects, direction):
    location = list(agent)

    if direction == 'up':
        location[0] -= 1
    elif direction == 'down':
        location[0] += 1
    elif direction == 'left':
        location[1] -= 1
    else:
        location[1] += 1

    if location[0] > 5 or location[0] < 1 or location[1] > 5 or location[1] < 1:
        return False

    for ob in objects:
        if tuple(location) == ob:
            return True

    return False

In [35]:
#This is the process for using the code above and solving board 1 of the assignment

#Create an empty board, a movable piece and an agent
board1 = Board()
t1 = Tpiece()
mike = PSA()

#Add things to the board/env as specified for board1 in the assignment
board1.add_thing(mike, (1, 1))
board1.add_thing(t1, (5, 2))

#Reach the first goal state (agent wants to go near the piece to move it) and save the resulting node class object to a variable
first_goal = mike(board1) #This executes the __call__ method of the agent which is where the whole process starts

In [38]:
#second board
board2 = Board()
t1 = Tpiece()
m1 = Mobs()
m2 = Mobs()
m3 = Mobs()
o1 = Oobs()
o2 = Oobs()
mike = PSA()

board2.add_thing(mike, (5, 1))
board2.add_thing(t1, (2, 3))
board2.add_thing(m1, (3, 2))
board2.add_thing(m2, (3, 3))
board2.add_thing(m3, (3, 4))
board2.add_thing(o1, (3, 1))
board2.add_thing(o2, (3, 5))

first_goal = mike(board2)

In [45]:
def printStateMatrix(state):
    temp = dict(test)
    
    for i in range(1, 8):
        for j in range(1, 8):
            if i == 1:
                if j == 1:
                    print('+ ', end = '')
                elif j == 7:
                    print('+')
                else:
                    print(str(j - 1) + ' ', end = '')
            
            elif i == 7:
                if j == 1:
                    print('+ ', end = '')
                elif j == 7:
                    print('+')
                else:
                    print(str(j - 1) + ' ', end = '')
            
            else:
                if j == 1:
                    print(str(i - 1) + ' ', end = '')
                elif j == 7:
                    print(str(i - 1))
                else:
                    if temp['A'] == (i - 1, j - 1):
                        print('A ', end = '')
                    elif temp['T'] == (i - 1, j - 1):
                        print('T ', end = '')
                    elif [locus for locus in list(temp['M']) if locus == (i - 1, j - 1)]:
                        print ('M ', end = '')
                    elif [locus for locus in list(temp['O']) if locus == (i - 1, j - 1)]:
                        print ('O ', end = '')
                    else:
                        print('- ', end = '')

In [46]:
printStateMatrix(first_goal.path()[-1].state)

+ 1 2 3 4 5 +
1 - - - - - 1
2 - - T - - 2
3 O M M M O 3
4 - - - - - 4
5 A - - - - 5
+ 1 2 3 4 5 +


In [47]:
len(first_goal.path())

11502

'pushRight'